In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam

# Sample larger input text corpus
corpus = [
    "Anna ,embrace change",
    "Change is constant",
    "Anna is a positive thinker",
    "Anna takes life as a challenge",
    "Anna is a fearless girl",
    "Ashi loves to defeat the obstacles"
]

C:\Users\SREJITA\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# Prepare tokenizer and word index
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
word_index = tokenizer.word_index
total_words = len(word_index) + 1
print("Word index",word_index)
print("total words",total_words)

Word index {'anna': 1, 'is': 2, 'a': 3, 'change': 4, 'embrace': 5, 'constant': 6, 'positive': 7, 'thinker': 8, 'takes': 9, 'life': 10, 'as': 11, 'challenge': 12, 'fearless': 13, 'girl': 14, 'ashi': 15, 'loves': 16, 'to': 17, 'defeat': 18, 'the': 19, 'obstacles': 20}
total words 21


In [4]:
# Prepare input-output pairs
sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        sequences.append(n_gram_sequence)

# Pad sequences and create predictors and label
max_sequence_len = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_sequence_len, padding='pre')
x_train, y_train = sequences[:,:-1], sequences[:,-1]

# One-hot encode the labels
y_train = tf.keras.utils.to_categorical(y_train, num_classes=total_words)
print("Xtrain",x_train)
print("ytrain",y_train)

Xtrain [[ 0  0  0  0  1]
 [ 0  0  0  1  5]
 [ 0  0  0  0  4]
 [ 0  0  0  4  2]
 [ 0  0  0  0  1]
 [ 0  0  0  1  2]
 [ 0  0  1  2  3]
 [ 0  1  2  3  7]
 [ 0  0  0  0  1]
 [ 0  0  0  1  9]
 [ 0  0  1  9 10]
 [ 0  1  9 10 11]
 [ 1  9 10 11  3]
 [ 0  0  0  0  1]
 [ 0  0  0  1  2]
 [ 0  0  1  2  3]
 [ 0  1  2  3 13]
 [ 0  0  0  0 15]
 [ 0  0  0 15 16]
 [ 0  0 15 16 17]
 [ 0 15 16 17 18]
 [15 16 17 18 19]]
ytrain [[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [5]:
# Build LSTM model
model = Sequential()
model.add(Embedding(total_words, 50, input_length=max_sequence_len-1))  # Embedding layer with 50 dimensions
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.01))

# Train the model
model.fit(x_train, y_train, epochs=500, verbose=1)

# Function to generate text with temperature
def generate_text(seed_text, next_words, model, tokenizer, max_sequence_len, temperature=1.0):
    output_text = seed_text
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([output_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)[0]
        predicted_probs = np.asarray(predicted_probs).astype('float64')
        predicted_probs = np.log(predicted_probs) / temperature
        exp_preds = np.exp(predicted_probs)
        predicted_probs = exp_preds / np.sum(exp_preds)
        predicted_word_index = np.random.choice(range(len(predicted_probs)), p=predicted_probs)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_word_index:
                output_word = word
                break
        output_text += " " + output_word
    return output_text

# Function to generate multiple sentences
def generate_sentences(seed_text, num_sentences, words_per_sentence, model, tokenizer, max_sequence_len, temperature=1.0):
    sentences = []
    for _ in range(num_sentences):
        sentence = generate_text(seed_text, words_per_sentence, model, tokenizer, max_sequence_len, temperature)
        sentences.append(sentence.strip())
    return ', '.join(sentences)



C:\Users\SREJITA\anaconda3\lib\site-packages\keras\src\layers\core\embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - loss: 3.0440
Epoch 2/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step - loss: 2.9947
Epoch 3/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 2.9056
Epoch 4/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step - loss: 2.7346
Epoch 5/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step - loss: 2.8364
Epoch 6/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - loss: 2.5300
Epoch 7/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - loss: 2.4670
Epoch 8/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - loss: 2.3692
Epoch 9/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step - loss: 2.1999
Epoch 10/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step - loss: 1.9857
Epoch 11/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - loss: 1.7635
Epoch 12/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - loss: 1.5394
Epoch 13/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - loss: 1.3530
Epoch 14/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - loss: 1.2037
Epoch 15/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - loss: 1.0531
Epoch 16/50

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - loss: 0.2532
Epoch 83/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step - loss: 0.2532
Epoch 84/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - loss: 0.2531
Epoch 85/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step - loss: 0.2531
Epoch 86/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - loss: 0.2531
Epoch 87/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - loss: 0.2531
Epoch 88/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step - loss: 0.2531
Epoch 89/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step - loss: 0.2531
Epoch 90/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - loss: 0.2531
Epoch 91/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - loss: 0.2530
Epoch 92/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - loss: 0.2530
Epoch 93/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step - loss: 0.2530
Epoch 94/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step - loss: 0.2530
Epoch 95/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 0.2530
Epoch 96/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - loss: 0.2530
Epoch 97/500


Epoch 163/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - loss: 0.2526
Epoch 164/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - loss: 0.2526
Epoch 165/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - loss: 0.2526
Epoch 166/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - loss: 0.2526
Epoch 167/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - loss: 0.2526
Epoch 168/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step - loss: 0.2526
Epoch 169/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - loss: 0.2526
Epoch 170/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step - loss: 0.2526
Epoch 171/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - loss: 0.2526
Epoch 172/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - loss: 0.2525
Epoch 173/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - loss: 0.2525
Epoch 174/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - loss: 0.2525
Epoch 175/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - loss: 0.2525
Epoch 176/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - loss: 0.2525
Epoch 177/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step - l

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step - loss: 0.2526
Epoch 244/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step - loss: 0.2524
Epoch 245/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step - loss: 0.2526
Epoch 246/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step - loss: 0.2524
Epoch 247/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - loss: 0.2525
Epoch 248/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 0.2524
Epoch 249/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step - loss: 0.2525
Epoch 250/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step - loss: 0.2524
Epoch 251/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step - loss: 0.2525
Epoch 252/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - loss: 0.2524
Epoch 253/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - loss: 0.2524
Epoch 254/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - loss: 0.2524
Epoch 255/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step - loss: 0.2524
Epoch 256/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step - loss: 0.2524
Epoch 257/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step - loss: 0.25

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - loss: 0.2523
Epoch 324/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - loss: 0.2523
Epoch 325/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - loss: 0.2523
Epoch 326/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step - loss: 0.2523
Epoch 327/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step - loss: 0.2523
Epoch 328/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step - loss: 0.2523
Epoch 329/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step - loss: 0.2523
Epoch 330/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step - loss: 0.2523
Epoch 331/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - loss: 0.2523
Epoch 332/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step - loss: 0.2523
Epoch 333/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - loss: 0.2523
Epoch 334/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - loss: 0.2523
Epoch 335/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - loss: 0.2523
Epoch 336/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step - loss: 0.2523
Epoch 337/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - loss: 0.25

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - loss: 0.2522
Epoch 404/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step - loss: 0.2522
Epoch 405/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - loss: 0.2522
Epoch 406/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - loss: 0.2522
Epoch 407/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step - loss: 0.2522
Epoch 408/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - loss: 0.2522
Epoch 409/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - loss: 0.2522
Epoch 410/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - loss: 0.2522
Epoch 411/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss: 0.2522
Epoch 412/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - loss: 0.2522
Epoch 413/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 0.2522
Epoch 414/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - loss: 0.2522
Epoch 415/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - loss: 0.2522
Epoch 416/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - loss: 0.2522
Epoch 417/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - loss: 0.2522
Epo

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 0.2522
Epoch 484/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step - loss: 0.2522
Epoch 485/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - loss: 0.2522
Epoch 486/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step - loss: 0.2522
Epoch 487/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step - loss: 0.2522
Epoch 488/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - loss: 0.2522
Epoch 489/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 0.2522
Epoch 490/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - loss: 0.2522
Epoch 491/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - loss: 0.2522
Epoch 492/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step - loss: 0.2522
Epoch 493/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 0.2522
Epoch 494/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 0.2522
Epoch 495/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - loss: 0.2522
Epoch 496/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - loss: 0.2522
Epoch 497/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step - loss: 0.2522

In [6]:
# Generate multiple sentences
seed_text = "Anna"
num_sentences = 5
words_per_sentence = 10
temperature = 1.0  # Adjust temperature for more or less randomness
generated_text = generate_sentences(seed_text, num_sentences, words_per_sentence, model, tokenizer, max_sequence_len, temperature)
print(generated_text)


Anna takes life as a challenge challenge challenge challenge the defeat, Anna is a fearless girl girl anna challenge positive thinker thinker, Anna is a fearless girl challenge obstacles thinker obstacles challenge obstacles, Anna takes life as a challenge challenge challenge challenge anna girl, Anna is a fearless girl thinker challenge anna a thinker girl
